# Featurestore - Churn Demo
## Feature Store Setup
Christos Aniftos

## Install required libraries

## Import modules

In [85]:
from google.cloud import aiplatform

In [86]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import FeaturestoreMonitoringConfig
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [88]:
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]

LOCATION = 'europe-west4' 
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "telco"
ENTITY="customer"

## Define clients for FS admin and data management

In [89]:
# Create admin_client for CRUD
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [91]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

Location: 	 projects/myfirstproject-226013/locations/europe-west4
Feature Store: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco
Entity: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco/entityTypes/customer
Feature: 	 projects/myfirstproject-226013/locations/europe-west4/featurestores/telco/entityTypes/customer/features/{}


In [68]:
# Create operation client to poll LRO status.
#lro_client = operations_v1.OperationsClient(admin_client.transport.grpc_channel)

## Create FeatureStore

In [92]:
new_feature_store = featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1 # The number of nodes for each cluster (bigtable)
            )
        )
try:
    print(
        admin_client.create_featurestore(
            featurestore_service_pb2.CreateFeaturestoreRequest(
                parent=LOC_PATH,
                featurestore_id=FEATURESTORE_ID,
                featurestore= new_feature_store
            )
        ).result()
    )
except Exception as ex:
    print(ex)

name: "projects/478111835512/locations/europe-west4/featurestores/telco"



 ## List all Feature Stores

In [93]:
list(admin_client.list_featurestores(parent=LOC_PATH))

[name: "projects/478111835512/locations/europe-west4/featurestores/universe"
 create_time {
   seconds: 1629721429
   nanos: 217408000
 }
 update_time {
   seconds: 1629721434
   nanos: 168242000
 }
 etag: "AMEw9yMFZ4RwfJ97zjS7rEryXofTsQ9eQSfIfks5kxyBcQEM_10Pf-DdvmObCt7CvOM="
 online_serving_config {
   fixed_node_count: 1
 }
 state: STABLE,
 name: "projects/478111835512/locations/europe-west4/featurestores/telco"
 create_time {
   seconds: 1629807079
   nanos: 493648000
 }
 update_time {
   seconds: 1629807079
   nanos: 569225000
 }
 etag: "AMEw9yMH3y0iHSAnyEHXGR1gMvklBGgq9ozHJVvY09K-byPU6FDM46Na-qn0_JJd62s1"
 online_serving_config {
   fixed_node_count: 1
 }
 state: STABLE]

## Get Feature store by name

In [94]:
try:
    feature_store_obj = admin_client.get_featurestore(name = FS_PATH)
except Exception as ex:
    print(ex)
feature_store_obj

name: "projects/478111835512/locations/europe-west4/featurestores/telco"
create_time {
  seconds: 1629807079
  nanos: 493648000
}
update_time {
  seconds: 1629807079
  nanos: 569225000
}
etag: "AMEw9yMqa2mWvSWHeo6eBY6soWehVWIGMfdRG8TWi0QaTSBxCSfq4c_wZbzPB7JSzbv4"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE

### Update feature store (changing online serving nodes)

In [72]:
feature_store_obj.online_serving_config.fixed_node_count=1
admin_client.update_featurestore(
    featurestore_service_pb2.UpdateFeaturestoreRequest(featurestore=feature_store_obj)
)

## Create a feature store Entity

In [95]:
try:
    entity_type_obj = admin_client.create_entity_type(
        featurestore_service_pb2.CreateEntityTypeRequest(
            parent=FS_PATH,
            entity_type_id=ENTITY, 
            entity_type=entity_type_pb2.EntityType(description="customer features"))).result()
    print(entity_type_obj)
except Exception as ex:
    print(ex)

name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer"
etag: "AMEw9yOBrPR-11UWd95fi8ZP0rlT8ARpZNbBA0K0shejPQkyfmQa"



In [96]:
type(entity_type_obj)

google.cloud.aiplatform_v1beta1.types.entity_type.EntityType

In [99]:
from google.protobuf import duration_pb2 as duration 

duration = duration.Duration()
duration.FromSeconds(seconds=3600)


fmc=FeaturestoreMonitoringConfig(
    snapshot_analysis = FeaturestoreMonitoringConfig.SnapshotAnalysis(disabled=False, monitoring_interval = duration))
fmc

snapshot_analysis {
  monitoring_interval {
    seconds: 3600
  }
}

In [100]:
entity_type_obj.monitoring_config= fmc
entity_type_obj = admin_client.update_entity_type(featurestore_service_pb2.UpdateEntityTypeRequest(
            entity_type=entity_type_pb2.EntityType(name=entity_type_obj.name)),
            )

## Create features in batch

In [101]:
features_list_tmp = []

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="gender",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="gender"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="weather customer is senior citizen ",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="senior_citizen"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="does customer have Partner",
                    monitoring_config= fmc,
                    labels=[("pii", ""),("approved", "")]),
                feature_id="partner"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="does customer have Dependents",
                    monitoring_config= fmc,
                    labels=[("pii", ""),("approved", "")]),
                feature_id="dependents"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.INT64,
                    description="how long he is a custormer for",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="tenure"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.BOOL,
                    description="is using PhoneService",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="phone_service"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has MultipleLines",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="multiple_lines"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has InternetService",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="internet_service"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has OnlineSecurity",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="online_security"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if customer has OnlineBackup",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="online_backup"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has device protection",
                    monitoring_config= fmc,
                    labels=[("no-pii", "")]),
                feature_id="device_protection"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if uses tech support",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="tech_support"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has steaming tv service",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", ""),("validated", "")]),
                feature_id="streaming_tv"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="if has steaming movies",
                    labels=[("no-pii", "")]),
                feature_id="streaming_movies"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="type of contract",
                    labels=[("no-pii", "")]),
                feature_id="contract"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="Opt in for PaperlessBilling",
                    labels=[("no-pii", "")]),
                feature_id="paperless_billing"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.STRING,
                    description="what is the payment method",
                    labels=[("no-pii", "")]),
                feature_id="payment_method"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="average monthly charges",
                    labels=[("no-pii", "")]),
                feature_id="monthly_charges"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="if male",
                    labels=[("no-pii", "")]),
                feature_id="male"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.BOOL,
                    description="if female",
                    labels=[("no-pii", "")]),
                feature_id="female"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="Monthly chrages Standartised -1 - 1",
                    labels=[("no-pii", "")]),
                feature_id="monthly_charges_std"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="Total Charges",
                    labels=[("no-pii", "")]),
                feature_id="total_charges"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE,
                    description="the average monthly spending of customer with similar profile",
                    labels=[("no-pii", "")]),
                feature_id="monthly_charges_avg_customer_clustered"))

    
    
try:
    features_list = admin_client.batch_create_features(
        parent=ENTITY_PATH,
        requests=features_list_tmp).result()
    print(features_list)
except Exception as ex:
    print(ex)

features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/gender"
  etag: "AMEw9yN_20Ry8NGA0ENN23dOmMTIoLocp4FTR9_5Xr0wsuDzZ_Ts"
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/senior_citizen"
  etag: "AMEw9yMwSg-Qij96IgSbUbUrhu7AP-gLxmnnVXBUqTViVx9iJ_aJ"
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/partner"
  etag: "AMEw9yPSqTB9o2A_JAVKfLDizIhyC8Bh_ihdSuulXheSUXn_XXH5"
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/dependents"
  etag: "AMEw9yMLu6PpwB1wCqoQTn-QErxwxM-26x1_Fg2kRpbwQmTevxSb"
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/tenure"
  etag: "AMEw9yP2Qn2CnWxRIdCYQSQQrXeq-BGvRAWsGIka3hnSnb2BXxVe"
}
features {
  name: "projects/478111835512/locat

## Update Feature

In [106]:
feature =admin_client.get_feature(name=FEATURE_PATH.format('monthly_charges'))
feature

name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/monthly_charges"
description: "average monthly charges"
value_type: DOUBLE
create_time {
  seconds: 1629807365
  nanos: 220284000
}
update_time {
  seconds: 1629807427
  nanos: 584239000
}
labels {
  key: "approved"
  value: ""
}
labels {
  key: "no-pii"
  value: ""
}
etag: "AMEw9yMbH35JDjw0Btn8zN43jdFhGjCY_k4BAL-l8istnl6h1D3DLrNjQAd7eo1Yyzl-"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
  }
}

In [107]:
tmp_list = list(feature.labels.items())
tmp_list.append(("bm", "12"))
feature.labels=tmp_list
feature.monitoring_config= fmc

In [108]:
admin_client.update_feature(
    featurestore_service_pb2.UpdateFeatureRequest(feature=feature)
)

name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/monthly_charges"
description: "average monthly charges"
value_type: DOUBLE
create_time {
  seconds: 1629807365
  nanos: 220284000
}
update_time {
  seconds: 1629807716
  nanos: 516584000
}
labels {
  key: "approved"
  value: ""
}
labels {
  key: "bm"
  value: "12"
}
labels {
  key: "no-pii"
  value: ""
}
etag: "AMEw9yMDiZ3BD1KuhhV1XrVQgEFKiEsJJ8zIl_Krcl3gnTeVXTJxNCA-Zf-CRUlqhI7s"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
  }
}

## Delete Feature

In [81]:
"""
try:
    admin_client.delete_feature(name=FEATURE_PATH.format('monthly_charges')).result()
except Exception as ex:
    print(ex)
"""

"\ntry:\n    admin_client.delete_feature(name=FEATURE_PATH.format('monthly_charges')).result()\nexcept Exception as ex:\n    print(ex)\n"

## Search Features

-  ``feature_id``: Supports = comparisons.
-  ``description``: Supports = comparisons. Multi-token filters should be enclosed in quotes.
-  ``entity_type_id``: Supports = comparisons.
-  ``value_type``: Supports = and != comparisons.
-  ``labels``: Supports key-value equality as well as key presence.
-  ``featurestore_id``: Supports = comparisons.

In [82]:
admin_client.search_features(location=LOC_PATH)

SearchFeaturesPager<features {
  name: "projects/478111835512/locations/europe-west4/featurestores/universe/entityTypes/customer/features/contract"
  description: "type of contract"
  create_time {
    seconds: 1629721439
    nanos: 280117000
  }
  update_time {
    seconds: 1629721439
    nanos: 280117000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/universe/entityTypes/customer/features/dependents"
  description: "does customer have Dependents"
  create_time {
    seconds: 1629721439
    nanos: 261458000
  }
  update_time {
    seconds: 1629721439
    nanos: 261458000
  }
  labels {
    key: "approved"
    value: ""
  }
  labels {
    key: "pii"
    value: ""
  }
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/universe/entityTypes/customer/features/device_protection"
  description: "if has device protection"
  create_time {
    seconds: 1629721439
    nanos: 2741770

In [83]:
q="featurestore_id={}".format(FEATURESTORE_ID)
admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=2, page_token="" ))

SearchFeaturesPager<features {
  name: "projects/478111835512/locations/europe-west4/featurestores/universe/entityTypes/customer/features/contract"
  description: "type of contract"
  create_time {
    seconds: 1629721439
    nanos: 280117000
  }
  update_time {
    seconds: 1629721439
    nanos: 280117000
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
features {
  name: "projects/478111835512/locations/europe-west4/featurestores/universe/entityTypes/customer/features/dependents"
  description: "does customer have Dependents"
  create_time {
    seconds: 1629721439
    nanos: 261458000
  }
  update_time {
    seconds: 1629721439
    nanos: 261458000
  }
  labels {
    key: "approved"
    value: ""
  }
  labels {
    key: "pii"
    value: ""
  }
}
next_page_token: "Cn4IgICAgICAuP4yEnBBTUV3OXlPbURVVkRnNXRSUjNQdmxtWC1ObVpDM3pIVDFCNFBSMVk2YjhtVGRSeFhuN1dXbmF1TFozbHdyOENPNlJNQnp6bG1QSC1FSDV0ZUpkeERXaVZvU3lBMXEyRnJlSlgzcmxva1R1WGs4dl9KGAI="
>

In [105]:
q="""
    feature_id=monthly_charges AND 
    value_type=DOUBLE AND 
    featurestore_id={}
""".format(FEATURESTORE_ID)

admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=5))

SearchFeaturesPager<features {
  name: "projects/478111835512/locations/europe-west4/featurestores/telco/entityTypes/customer/features/monthly_charges"
  description: "average monthly charges"
  create_time {
    seconds: 1629807365
    nanos: 220284000
  }
  update_time {
    seconds: 1629807427
    nanos: 584239000
  }
  labels {
    key: "approved"
    value: ""
  }
  labels {
    key: "no-pii"
    value: ""
  }
}
>